In [2]:
import requests 
import numpy as np
import pandas as pd
import json
import pprint

In [3]:
def get_json(npi="",first="",last="",tax = "",state = "",limit = 10,npiType=""):
    url = "https://npiregistry.cms.hhs.gov/api/?number=" + str(npi)+"&enumeration_type="+npiType+"&taxonomy_description=" + tax + "&name_purpose=&first_name=" + first + "&use_first_name_alias=&last_name=" + last + "&organization_name=&address_purpose=&city=&state="+state+"&postal_code=&country_code=&limit="+str(limit)+"&skip=&pretty=on&version=2.1"
    json = requests.get(url).json()
    return json
print(get_json(tax="Thoracic Surgery (Cardiothoracic Vascular Surgery)",limit=100,state ="NY"))

{'result_count': 100, 'results': [{'created_epoch': '1139344872000', 'enumeration_type': 'NPI-1', 'last_updated_epoch': '1272288420000', 'number': '1881666311', 'addresses': [{'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'MAILING', 'address_type': 'DOM', 'address_1': '214 CORNELIA ST', 'address_2': 'SUITE 201', 'city': 'PLATTSBURGH', 'state': 'NY', 'postal_code': '129012306', 'telephone_number': '518-562-7993', 'fax_number': '518-562-7064'}, {'country_code': 'US', 'country_name': 'United States', 'address_purpose': 'LOCATION', 'address_type': 'DOM', 'address_1': '214 CORNELIA ST', 'address_2': 'SUITE 201', 'city': 'PLATTSBURGH', 'state': 'NY', 'postal_code': '129012306', 'telephone_number': '518-562-7993', 'fax_number': '518-562-7064'}], 'practiceLocations': [], 'basic': {'first_name': 'ALBERT', 'last_name': 'ABBOTT', 'middle_name': 'E', 'credential': 'MD', 'sole_proprietor': 'NO', 'gender': 'M', 'enumeration_date': '2006-02-07', 'last_updated': '2010-04-26

In [ ]:
pprint.pprint(get_json(first="brian",last="neri"))

{'result_count': 1,
 'results': [{'addresses': [{'address_1': '1530 FRONT ST',
                             'address_purpose': 'LOCATION',
                             'address_type': 'DOM',
                             'city': 'EAST MEADOW',
                             'country_code': 'US',
                             'country_name': 'United States',
                             'postal_code': '115542265',
                             'state': 'NY',
                             'telephone_number': '516-758-8670'},
                            {'address_1': '2800 MARCUS AVE',
                             'address_purpose': 'MAILING',
                             'address_type': 'DOM',
                             'city': 'NEW HYDE PARK',
                             'country_code': 'US',
                             'country_name': 'United States',
                             'postal_code': '110421008',
                             'state': 'NY',
                             'telepho

In [ ]:
"""
Some comments about the above json response

1) The Addresses array has two occurrences. The first address in the array will always be the 
Primary Practice Location and the second address in the array will always be the Mailing Address.

2) 
"""

'\nSome comments about the above json response\n\n1) The Addresses array has two occurrences. The first address in the array will always be the \nPrimary Practice Location and the second address in the array will always be the Mailing Address.\n\n2) \n'

In [ ]:
print(get_json(tax="Pediatric Orthopaedic Surgery",state="NY",limit=200,npiType = "NPI-1")['result_count'])
pprint.pprint(get_json(tax="Pediatric Orthopaedic Surgery",state="NY",npiType = "NPI-2",limit=200)['results'][0])

84
{'addresses': [{'address_1': '2233 STATE ROUTE 86',
                'address_purpose': 'LOCATION',
                'address_type': 'DOM',
                'city': 'SARANAC LAKE',
                'country_code': 'US',
                'country_name': 'United States',
                'fax_number': '518-897-2423',
                'postal_code': '129835644',
                'state': 'NY',
                'telephone_number': '518-897-2636'},
               {'address_1': 'PO BOX 471',
                'address_purpose': 'MAILING',
                'address_type': 'DOM',
                'city': 'SARANAC LAKE',
                'country_code': 'US',
                'country_name': 'United States',
                'fax_number': '518-897-2423',
                'postal_code': '129830471',
                'state': 'NY',
                'telephone_number': '518-897-2363'}],
 'basic': {'authorized_official_first_name': 'JULIE',
           'authorized_official_last_name': 'WOOD',
           'authorized

In [ ]:
"""
Attributes of interest
1) address_1 (this attribute will always be the primary practice location )
2) last_updated (in the "basic" key)
3) taxonomies (provider may have multiple taxonomies and figure out how it works with the tree.)
"""

'\nAttributes of interest\n1) address_1 (this attribute will always be the primary practice location )\n2) last_updated (in the "basic" key)\n3) taxonomies (provider may have multiple taxonomies and figure out how it works with the tree.)\n'

In [5]:
"""
Task: make a pandas dataFrame out of the json response with key 
attributes for every npi within a subset of providers sorted by taxonomy
(i.e. get all Pedreatric Orthopedreic surgeons in NY with NPI-1 type number)

Attribues: 
- NPI number 
- Name (first and last)
- primary practice location 
- additional practice location(may be multiple)
- last updated epoch/timestamp

"""
#get entire results array. its elements are dictionaries
results = get_json(tax="Pediatric Orthopaedic Surgery",state="NY",npiType = "NPI-1",limit=200)['results']
#loop through every result dictionary and append information to info 

In [6]:
agg = []
for result in results: 

    info = [] #(npi,address_number)
    #get npi number and add to info
    info.append(result["number"])
    
    #get first and last name and add to info
    info.append(result["basic"]["first_name"])
    info.append(result["basic"]["last_name"])

    #should loop through all the indicies since some entries may be blank
    
    #get primary and other practice location in dictionary form and put in "addresses" list
    #Aside: seems like the api documentation is not accurate as the first entry is not always the location address, so we need to handle that 
    primary = result["addresses"][:2]
    
    for i in range(len(primary)): 
      if result["addresses"][i]['address_purpose'] != "LOCATION": 
        primary.pop(i)
    


    addresses = primary + result["practiceLocations"]
    #addresses = result["practiceLocations"]
    info.append(addresses)

    
    #append information for one provider to aggregate list
    agg.append(info)

In [ ]:
#agg[10:20]

In [8]:
col = ["NPI","FIRST NAME", "LAST NAME","PRACTICE ADDRESSES"]
df = pd.DataFrame(agg,columns=col)

In [9]:
#final dataframe with all pediatric ortho surgeons in NY with all practice location addresses. 
#practice addresses column are a list of dictionaries
df

,NPI,FIRST NAME,LAST NAME,PRACTICE ADDRESSES
0,1952305369,STEPHEN,ALBANESE,"[{'country_code': 'US', 'country_name': 'Unite..."
1,1871770255,ABIGAIL,ALLEN,"[{'country_code': 'US', 'country_name': 'Unite..."
2,1376710178,JAMES,BARSI,"[{'country_code': 'US', 'country_name': 'Unite..."
3,1598024044,ALLISON,BINKLEY,"[{'country_code': 'US', 'country_name': 'Unite..."
4,1679527402,CHARLES,BROWN,"[{'country_code': 'US', 'country_name': 'Unite..."
...,...,...,...,...
79,1114198751,MATTHEW,WAGNER,"[{'country_code': 'US', 'country_name': 'Unite..."
80,1669454625,ANDREW,WEILAND,"[{'country_code': 'US', 'country_name': 'Unite..."
81,1649298977,ROGER,WIDMANN,"[{'country_code': 'US', 'country_name': 'Unite..."
82,1376791277,HENOCK,WOLDE-SEMAIT,"[{'country_code': 'US', 'country_name': 'Unite..."


In [ ]:
!pip install datasketch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 1.6 MB/s eta 0:00:00


In [ ]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest

address = df.loc[:,"PRACTICE ADDRESSES"]
address_set = []
for i in address:
    for j in i:
        address_set.append (list(j.values()))

hashed = []
th=0.6 # setting threshold
lsh = MinHashLSH(threshold=th, num_perm=128)
lsh_forest = MinHashLSHForest(num_perm=128)
for i in range(len(address_set)):
    m = MinHash(num_perm=128)
    for j in address_set[i]:
        m.update(j.encode('utf8'))
    lsh_forest.add(str(i), m)
    lsh.insert(str(i), m)
    hashed.append(m)

lsh_forest.index()

target = 10 # change this to decide which index we are trying to compare.
#LSH does not work as well compared to LSH forest
lsh_result = lsh.query(hashed[target])
print("LSH: approximate neighbours with Jaccard similarity > " + str(th), lsh_result)
print(len(lsh_result))
for i in lsh_result:
    print(address_set[int(i)])

# can use this for scoring
top_how_many = 5 # how many closest do we want (not necessarily ordered correctly)
lsh_forest_result = lsh_forest.query(hashed[target], top_how_many)
print("LSH forest result:" + str(lsh_forest_result))
for i in lsh_forest_result:
    print(address_set[int(i)])
    
# https://ekzhu.com/datasketch/lshensemble.html#minhash-lsh-ensemble I think this talks about containment may be useful for multiple addresses

LSH: approximate neighbours with Jaccard similarity > 0.6 ['10', '48']
2
['US', 'United States', 'LOCATION', 'DOM', '301 E 17TH ST', 'NEW YORK', 'NY', '100033804', '212-598-6261', '646-794-1201']
['US', 'United States', 'LOCATION', 'DOM', '301 E 17TH ST', 'NYU HOSPITAL FOR JOINT DISEASES SUITE 413', 'NEW YORK', 'NY', '100033804', '212-598-6403', '212-598-6084']
LSH forest result:['34', '42', '58', '106', '10']
['US', 'United States', 'LOCATION', 'DOM', '535 E 70TH ST', 'NEW YORK', 'NY', '100214823', '212-606-1631', '212-774-2776']
['US', 'United States', 'LOCATION', 'DOM', '535 E 70TH ST', 'NEW YORK', 'NY', '100214823', '212-774-2577']
['US', 'United States', 'LOCATION', 'DOM', '535 E 70TH ST', 'NEW YORK', 'NY', '100214823', '212-774-2111']
['US', 'United States', 'LOCATION', 'DOM', '535 E 70TH ST', 'NEW YORK', 'NY', '100214823', '212-606-1325']
['US', 'United States', 'LOCATION', 'DOM', '301 E 17TH ST', 'NEW YORK', 'NY', '100033804', '212-598-6261', '646-794-1201']


In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy 
import pandas

def get_locations(first, last):
    """
    Returns a list of addresses for a given provider with first and last name as parameters
    Limits search for providers in New York with a speciality in Pediatric Orthopedic Surgery
    """

    search = first+ "+" +last + "+Pediatric+Orthopedic+Surgery+New+York+Healthgrades&oq="+first+"+"+last+"+Pediatric+Orthopedic+Surgery+New+York+Healthgrades"
    url = 'https://www.google.com/search'

    headers = {
        'Accept' : '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
    }
    parameters = {'q': search}

    content = requests.get(url, headers = headers, params = parameters).text
    soup = BeautifulSoup(content, 'html.parser')

    search = soup.find(id = 'search')
    first_link = search.find('a')['href']

    #if healthgrades is not the link, then just return an empty addresses list 
    if not "www.healthgrades.com" in first_link:
      return ([],"not healthgrades")

    #get text content from a specific div
    response = requests.get(first_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    divs = soup.find_all('div', class_='office-title')

    addresses = []
    for i in divs:
        #find all span tags in the div
        span = i.find_all('span')
        address = [''.join(i.text) for i in span]
        addresses.append(address)


    if addresses == []:
      spans = soup.find_all('span', class_='hg-track')
      for i in spans:
        #find all span tags in the div
        span = i.find_all('span')
        address = [''.join(i.text) for i in span]

        #get only spans with address info, not organization name
        if len(address) ==5:
          address = address[1:]
        #NEED TO HANDLE THE CASE WHERE ORG NAME DOESNT EXIST BETTER 

        addresses.append(address)

      #get rid of any empty lists in addresses
      addresses = list(filter(None, addresses))
      
    
    return (addresses, first_link)

In [ ]:
healthgrades_practice_addresses = []
#df["HEALTHGRADES ADDRESSES"] = 

for index, row in df.iterrows():
  if index <20:
    first = row["FIRST NAME"]
    last = row["LAST NAME"]
    print(first + ", " + last)
    result = get_locations(first,last)
    healthgrades_practice_addresses.append(result[0])
    print("link: " + result[1])
    #df[""]

healthgrades_practice_addresses

#80th entry does not produce healthgrades result because no healthgrades page for ANDREW WEILAND. 
#Need to handle exceptions properly, -> what do we do? 

#Also need to handle some healthgrade pages being slightly different in html -> need to account for all cases somehow

STEPHEN, ALBANESE
link: https://www.healthgrades.com/physician/dr-stephen-albanese-x9xfv
ABIGAIL, ALLEN
link: https://www.healthgrades.com/physician/dr-abigail-allen-ycyqx
JAMES, BARSI
link: https://www.healthgrades.com/physician/dr-james-barsi-xpnxq
ALLISON, BINKLEY
link: https://www.healthgrades.com/physician/dr-allison-binkley-c37bz
CHARLES, BROWN
link: https://www.healthgrades.com/physician/dr-charles-brown-xwjq9
CORDELIA, CARTER
link: https://www.healthgrades.com/physician/dr-cordelia-carter-xvn6c
PABLO, CASTANEDA
link: https://www.healthgrades.com/physician/dr-pablo-castaneda-xyn5j3g
MAURO, CATALETTO
link: https://www.healthgrades.com/physician/dr-mauro-cataletto-xw945
CHRISTOPHER, CHAPMAN
link: https://www.healthgrades.com/physician/dr-christopher-chapman-c2ljxui288
ALICE, CHU
link: https://www.healthgrades.com/physician/dr-alice-chu-2ycr7
STRUAN, COLEMAN
link: https://www.healthgrades.com/physician/dr-struan-coleman-xh8kg
P., COOK
link: https://www.healthgrades.com/physician/dr

[[['6620 Fly Rd ', 'East Syracuse, ', 'NY ', '13057']],
 [['5 E 98th St Fl 9 ', 'New York, ', 'NY ', '10029']],
 [['14 Technology Dr Ste 12 ', 'East Setauket, ', 'NY ', '11733']],
 [['818 Ellicott St ', 'Buffalo, ', 'NY ', '14203'],
  ['4949 Harlem Rd ', 'Amherst, ', 'NY ', '14226'],
  ['5959 Big Tree Rd Ste 108 ', 'Orchard Park, ', 'NY ', '14127']],
 [['34 S Bedford Rd ', 'Mount Kisco, ', 'NY ', '10549']],
 [['20 York St ', 'New Haven, ', 'CT ', '06510']],
 [['301 E 17th St Ofc 400B ', 'New York, ', 'NY ', '10003']],
 [['200 Old Country Rd Ste 470 ', 'Mineola, ', 'NY ', '11501']],
 [['450 Clarkson Ave ', 'Brooklyn, ', 'NY ', '11203']],
 [['140 Bergen St ', 'Newark, ', 'NJ ', '07103'],
  ['301 E 17th St # 400 ', 'New York, ', 'NY ', '10003'],
  ['218 Ridgedale Ave Ste 202 ', 'Cedar Knolls, ', 'NJ ', '07927']],
 [['146 Birch Hill Rd ', 'Locust Valley, ', 'NY ', '11560'],
  ['1200 Constitution Ave ', 'Philadelphia, ', 'PA ', '19112'],
  ['519 E 72nd St Ste 203 ', 'New York, ', 'NY ', '10

## Implementing a model that searches Google API, Webscraping Libraries, and NLP lmodels

In [ ]:
from googlesearch import search
from bs4 import BeautifulSoup
import spacy
import re

In [ ]:
!python -m spacy download en_core_web_sm

2023-05-07 19:25:45.932412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
url = 'https://doctors.stonybrookmedicine.edu/provider/James%20M%20Barsi/2250698#provider-details-experience'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
#html_content = soup.get_text()
#print(html_content)
#text = ' '.join(html_content.split())
text = ' '.join(soup.stripped_strings)
print(text)

Request unsuccessful. Incapsula incident ID: 8222000620129981578-192248625107963786


In [ ]:
# Load the spaCy model for NER
nlp = spacy.load('en_core_web_lg')

OSError: ignored

In [ ]:
"""
# Define the website URL
url = 'https://doctors.stonybrookmedicine.edu/provider/James%20M%20Barsi/2250698#provider-details-experience'

# Retrieve the HTML content of the website
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
html_content = soup.get_text()
print(html_content)

"""
# Perform named entity recognition on the HTML content
doc = nlp(text)
addresses = []

print(doc.ents)

# Extract entities of type "ADDRESS"
for ent in doc.ents:
    if ent.label_ == 'ADDRESS':
        print("in if")
        addresses.append(ent.text)
print(addresses)

# Filter and extract address information using regular expressions
for address in addresses:
    print("ran")
    # Remove any leading/trailing whitespace
    address = address.strip()

    # Remove any phone numbers or email addresses
    address = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', '', address)
    address = re.sub(r'\(?[0-9]{3}\)?[-.\s]?[0-9]{3}[-.\s]?[0-9]{4}', '', address)

    # Check if the address contains a zip code
    if re.search(r'\b\d{5}(?:[-\s]\d{4})?\b', address):
        print(address)
        break